In [91]:
import numpy as np
import pandas as pd
import sympy
from ast import literal_eval
import os, time

# Random Functions

In [92]:
RESULT_DIR = "final_results"
benchmark = "rational_2"

In [93]:
df = {}
functions = None
for f_name in os.listdir(RESULT_DIR):
    try:
        benchmark_name = None
        if len(f_name.split("_")) == 4:
            benchmark_name = f_name.split("_")[1]
        elif len(f_name.split("_")) == 5:
            benchmark_name = "_".join(f_name.split("_")[1:-2])
                
        tool_name =  f_name.split("_")[-2]
        if benchmark_name == benchmark:
            df[tool_name] = pd.read_csv(os.path.join(RESULT_DIR, f_name)).sort_values('index')
            
            if tool_name == 'volesti':
                try:
                    df['volesti']['details'] = df['volesti']['details'].apply(literal_eval)
                                        
                    df['volesti']['np'] = df['volesti']['details'].apply(lambda x: np.asarray([i['output'] for i in x]))
                    df['volesti']['output'] = list(zip(df['volesti']['np'].apply(np.mean), df['volesti']['np'].apply(np.std)))            
                
                    df['volesti']['np'] = df['volesti']['details'].apply(lambda x: np.asarray([i['time'] for i in x]))
                    df['volesti']['time'] = df['volesti']['np'].apply(np.mean)            
                except:
                    pass           
                 
            if tool_name == 'faza':
                # try:
                df['faza']['details'] = df['faza']['details'].apply(literal_eval)
                df['faza']['n_hrects'] = df['faza']['details'].apply(lambda x: x['logs'][0]['hrect_checked_num'] if 'logs' in x else x['output'][-1]['hrect_checked_num'])
                # except:
                #     pass

    except Exception as e:
        print(e)

list index out of range


In [94]:
tools = sorted(list(df.keys()))
tool0 = tools[0]
tool1 = tools[1]

df_res = df[tool0].join(df[tool1], on='index', lsuffix=f'_{tool0}', rsuffix=f'_{tool1}', how='outer')

for t in tools[2:]:
    df_res = df_res.join(df[t].add_suffix(f'_{t}'), on='index', how='outer')


In [95]:
df_res.columns

Index(['index', 'bechmark_faza', 'formula_faza', 'index_faza', 'output_faza',
       'error_faza', 'time_faza', 'details_faza', 'n_hrects', 'bechmark_gubpi',
       'formula_gubpi', 'bounds', 'index_gubpi', 'output_gubpi', 'error_gubpi',
       'time_gubpi', 'details_gubpi', 'bechmark_latte', 'formula_latte',
       'bounds_latte', 'index_latte', 'output_latte', 'error_latte',
       'time_latte', 'details_latte', 'bechmark_volesti', 'formula_volesti',
       'bounds_volesti', 'index_volesti', 'output_volesti', 'error_volesti',
       'time_volesti', 'details_volesti', 'np_volesti'],
      dtype='object')

In [96]:
df_res = df_res.sort_values('index_gubpi')
selected_cols = [f'formula_gubpi']
selected_cols.extend(['bounds'])
# selected_cols.extend([f'bounds_{t}' for t in tools])


for t in tools:
    selected_cols.append(f"output_{t}")
    df_res[f"output_{t}"] = df_res[f"output_{t}"].fillna(df_res[f"error_{t}"]).fillna('TBD').astype(str).str[:1000]
    selected_cols.append(f"time_{t}")
    df_res[f"time_{t}"] = df_res[f"time_{t}"].fillna(0).apply(lambda x:round(x, 3))
    if t == 'faza': 
        selected_cols.append(f"n_hrects")
    
df_res = df_res[selected_cols]
df_res['formula_gubpi'] = df_res['formula_gubpi'].apply(sympy.parse_expr)
df_res

,formula_gubpi,bounds,output_faza,time_faza,n_hrects,output_gubpi,time_gubpi,output_latte,time_latte,output_volesti,time_volesti
0,(1.03*x**2 + 0.48*x*y + 8.88*x + 1.1*y**2 + 1....,"[[0.1, 1], [0.1, 1]]","(0.5898230890246405, 0.6896906251302616)",272.643,1284,"(0.711266057983094, 0.8710624008180216)",0.766,Not a monomial: (5381801554707743/4 + (2091359...,0.050,"(0.648784, 0.03218630839036996)",0.070
1,(6.82*x**2 + 1.32*x*y + 8.69*x + 5.6*y**2 + 8....,"[[0.1, 1], [0.1, 1]]","(1.7129047384964033, 1.812863288989478)",589.094,5336,"(1.5639518246240296, 2.8058999441535044)",0.756,Not a monomial: (2488238794122199/8 + (1151232...,0.041,"(1.744203, 0.08964754620735582)",0.070
2,(5.72*x**2 + 7.83*x*y + 4.6*x + 3.43*y**2 + 8....,"[[0.1, 1], [0.1, 1]]","(0.7618363752627153, 0.8617597761952512)",249.278,847,"(0.7126014688609542, 1.305862755213216)",0.799,Not a monomial: (9457559217478042.0 + (5944751...,0.039,"(0.8220304, 0.052084158826268864)",0.065
3,(5.79*x**2 + 2.65*x*y + 4.62*x + 1.86*y**2 + 1...,"[[0.1, 1], [0.1, 1]]","(0.3981118938485614, 0.4980876414244374)",266.304,1061,"(0.5084030815642594, 0.6028950703253301)",0.627,Not a monomial: (8219069319951155/2 + (3670433...,0.043,"(0.4513803, 0.025515351967198098)",0.063
4,(7.82*x**2 + 7.55*x*y + 3.35*x + 5.83*y**2 + 8...,"[[0.1, 1], [0.1, 1]]","(0.7253642810024395, 0.825341176037968)",486.233,3929,"(0.8289588063133213, 1.0884849667935454)",0.637,Not a monomial: (21910012187157464.0 + (776870...,0.043,"(0.7749078, 0.039051415861143884)",0.061
5,(9.01*x**2 + 1.08*x*y + 4.7*x + 2.58*y**2 + 0....,"[[0.1, 1], [0.1, 1]]","(0.429674191347774, 0.5296126224002398)",300.136,1492,"(0.5274434402103584, 0.6621698378850945)",0.621,Not a monomial: (4458563631096791.0 + (4638707...,0.045,"(0.492859, 0.025273920198497106)",0.064
6,(3.48*x**2 + 7.62*x*y + 1.28*x + 0.49*y**2 + 8...,"[[0.1, 1], [0.1, 1]]","(0.5297946239357543, 0.6297896317555237)",265.410,1055,"(0.5438271679604468, 0.9204038520182458)",0.700,Not a monomial: (6980579422424269/4 + (5787125...,0.038,"(0.5966868, 0.040498774568621214)",0.067
7,(7.89*x**2 + 9.21*x*y + 1.62*x + 3.61*y**2 + 7...,"[[0.1, 1], [0.1, 1]]","(1.408511963280665, 1.508506265667054)",3168.302,36996,"(1.1161111672108006, 2.4764386805304133)",0.620,Not a monomial: (5764607523034235/256 + (52917...,0.054,"(1.452901, 0.09511290937091559)",0.061
8,(3.53*x**2 + 9.9*x*y + 8.84*x + 4.24*y**2 + 2....,"[[0.1, 1], [0.1, 1]]","(0.9676485370584922, 1.0675116365090673)",282.556,1235,"(0.8776772171648857, 1.6319422485166692)",0.646,Not a monomial: (7352126391682335.0 + (3670433...,0.047,"(0.9985051, 0.04657369069217083)",0.064
9,(4.49*x**2 + 9.55*x*y + 1.59*x + 0.72*y**2 + 9...,"[[0.1, 1], [0.1, 1]]","(1.7291135333996104, 1.8291130292403135)",10969.145,134834,"(1.1650928203898223, 2.915243659132659)",0.637,Not a monomial: (5764607523034235/256 + (38562...,0.041,"(1.817597, 0.1441395147105748)",0.064


In [97]:
df_res.to_csv(os.path.join(RESULT_DIR, f'result_{benchmark}_{int(time.time())}.csv'), index=False)